# Numerics for constraints on $C_7$ and $C_7'$

This notebook runs the numerics for constraints on the Wilson coeffiencts $C_7$ and $C_7'$.

It is based on a [notebook](https://github.com/DavidMStraub/paper-bvgamma-ps) provided as compagnion script to the paper [arXiv:1608.02556](https://arxiv.org/abs/1608.02556) by Ayan Paul and David M. Straub.

It requires [flavio](https://flav-io.github.io/) v0.21.3 or later.

## Preliminaries

Loading packages

In [1]:
import pickle
from functools import partial
from collections import OrderedDict
import flavio
import flavio.plots
import flavio.statistics.fits

Load the LCSR-only $B\to V$ form factors from [arXiv:1503.05534](https://arxiv.org/abs/1503.05534) (instead of the combined LCSR-LQCD fit used by default; see appendix A)

In [2]:
flavio.physics.bdecays.formfactors.b_v.bsz_parameters.bsz_load_v2_lcsr(flavio.default_parameters)

Relevant observables

In [3]:
observables = [
  'BR(B+->K*gamma)',
  'BR(B->Xsgamma)',
  'BR(B0->K*gamma)',
  'BR(Bs->phigamma)',
  'ADeltaGamma(Bs->phigamma)',
  'S_K*gamma',
  ('<ATIm>(B0->K*ee)', 0.002, 1.12),
  ('<P1>(B0->K*ee)', 0.002, 1.12),
]

## Setting up the fits

Defining the `FastFit` instances for a given Wilson coefficient scenario and observable set. See [the documentation](https://flav-io.github.io/docs/fits.html) for details.

In [4]:
def wc_fct(C7Re, C7Im, C7pRe, C7pIm):
    return { 'C7eff_bs': C7Re + 1j * C7Im, 'C7effp_bs': C7pRe + 1j * C7pIm }

In [5]:
def fastfit_obs(name, obslist):
    return flavio.statistics.fits.FastFit(
                name = name,
                nuisance_parameters = [p for p in flavio.default_parameters.all_parameters if p not in ['alpha_s', 'alpha_e', 'm_Z']],
                observables = obslist,
                fit_wc_function = wc_fct,
                input_scale = 4.8,
            )

In [6]:
fits = OrderedDict()
fits['global'] = observables
fits['BR'] = ['BR(B+->K*gamma)', 'BR(B->Xsgamma)', 'BR(B0->K*gamma)', 'BR(Bs->phigamma)',]
fits['A'] = ['ADeltaGamma(Bs->phigamma)']
fits['P1'] = [('<P1>(B0->K*ee)', 0.002, 1.12)]
fits['S'] = ['S_K*gamma']
fits['ATIm'] = [('<ATIm>(B0->K*ee)', 0.002, 1.12)]
fits['no_BR'] = [
  'ADeltaGamma(Bs->phigamma)',
  'S_K*gamma',
  ('<ATIm>(B0->K*ee)', 0.002, 1.12),
  ('<P1>(B0->K*ee)', 0.002, 1.12),
]
fits['BR(B+->K*gamma)'] = ['BR(B+->K*gamma)']
fits['BR(B0->K*gamma)'] = ['BR(B0->K*gamma)']
fits['BR(B->Xsgamma)'] = ['BR(B->Xsgamma)']
fits['BR(Bs->phigamma)'] = ['BR(Bs->phigamma)']

Generating the pseudo measurements for all the fits.

In [7]:
%%time
obs_fastfits={}
for k, v in fits.items():
    obs_fastfits[k] = fastfit_obs('C7-C7p fit '+ k, v)
    obs_fastfits[k].make_measurement(N=200, threads=4)

CPU times: user 4.57 s, sys: 500 ms, total: 5.07 s
Wall time: 3min 44s


## Running the fits

In [8]:
def ll(x, f):
    return obs_fastfits[f].log_likelihood([x[0], 0, x[1], 0])
    
ll_dict = {key:partial(ll, f=key) for key in fits.keys()}

def get_contour_data(f):
    x_min=-0.25
    x_max=+1.05
    y_min=-0.65
    y_max=+0.65
    if f == 'global':
        n_sigma = (1,2)
        steps = 50
    else:
        n_sigma = 1
        steps = 50
    return flavio.plots.likelihood_contour_data(ll_dict[f],
                       x_min, x_max, y_min, y_max, n_sigma=n_sigma,
                       steps=steps, threads=4)

In [9]:
%%time
data = OrderedDict()
for i, f in enumerate(fits):
    if f == 'ATIm':
        continue # no need to plot ATIm
    data[f] = get_contour_data(f)
with open('./data.p', 'wb') as f:
    pickle.dump((data,fits), f)

CPU times: user 1.09 s, sys: 422 ms, total: 1.51 s
Wall time: 33min 17s
